In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
import cvxpy as cvx
import numpy as np

In [ ]:
a = 0.33
m1 = m3 = 0.7
m2 = 0.34

In [ ]:

def compute_draining_times(w):
    workload_1, workload_2 = compute_workloads(arrival_buffer,inter_buffer,drain_buffer)
    drain_time_1= workload_1/(1-mu_demand*2/mu_drain)
    drain_time_2 = workload_2/(1-mu_demand/mu_fast)
    return drain_time_1, drain_time_2

In [ ]:
#a = 0
workload_mat = np.array([[2/m1,1/m3,1/m3],[1/m2,1/m2,0]])
load = np.array([[2*a/m1],[a/m2]])
num_buffers = workload_mat.shape[1]

In [ ]:
1/(1-load)

In [ ]:
drain_time = (workload_mat @ x)/(1-load)
drain_time

In [ ]:
workload_mat @ x

In [ ]:
buffer_processing_matrix = np.array([[-m1,0,0],[+m1,-m2,0],[0,+m2,-m3]])

In [ ]:
cost_per_buffer = np.array([1.5,1,2])[:,np.newaxis]
#cost_per_buffer = np.array([1,1.001,1])[:,np.newaxis]
cost_vec = cost_per_buffer.T @ buffer_processing_matrix

In [ ]:
x_init = np.array([0,0,10000])[:,np.newaxis]
x_init = np.array([5000,0,0])[:,np.newaxis]
x_init = np.array([0,5000,0])[:,np.newaxis]
x_init = np.array([2500,0,3000])[:,np.newaxis]
horizon = 2000

In [ ]:
resource_1 = np.array([1,0,1])
resource_2 = np.array([0,1,0])

In [ ]:
demand_rate = np.array([a,0,0])[:,np.newaxis]

In [ ]:
kappa_w = 0#1e3

In [ ]:
min_drain_time

In [ ]:

min_drain_time = max((workload_mat @ x)/(1-load))

In [ ]:
min_drain_time/h

In [ ]:
int(min_drain_time/horizon)

In [ ]:
(workload_mat @ x_init)/(1-load)

In [ ]:
workload_mat[0] @ buffer_processing_matrix

In [ ]:
state_seq = []
cost_seq = []
action_seq = []

original_min_drain_time = max((workload_mat @ x_init)/(1-load))

x = x_init
cost = 0
activate = False

w = workload_mat @ x

x_optim = cvx.Variable((num_buffers, 1),nonneg=True)
objective = cvx.Minimize(cost_per_buffer.T @ x_optim)# + lam*cvx.sum(workload_mat @ x - w))
constraints = [workload_mat[0] @ x_optim >= w[0],
               workload_mat[1] @ x_optim >= w[1],
                x_optim >= np.zeros((num_buffers, 1)),
              #1/(1-load[1]) * workload_mat[1] @ x_optim <= original_min_drain_time,
              ]#,cvx.sum(x) <= np.sum(x0)]
prob = cvx.Problem(objective, constraints)
eff_cost = prob.solve(solver=eval('cvx.ECOS'))
x_star = x_optim.value
w_star = workload_mat @ x_star

min_drain_time = max((w_star)/(1-load))
#min_drain_time = original_min_drain_time

for j in range(int(min_drain_time/horizon)):
    #print(j/int(min_drain_time/horizon))
    
    w = workload_mat @ x
    print(w.ravel().astype(int))
    print((w-(1-load)*horizon).ravel())
    updated_min_drain_time = original_min_drain_time - j*horizon
    min_drain_time = max((w)/(1-load))
    print((w/(1-load)).ravel(),original_min_drain_time - (j+1)*horizon)
    #print(min_drain_time/original_min_drain_time)
    
        
    x_optim = cvx.Variable((num_buffers, 1),nonneg=True)
    objective = cvx.Minimize(cost_per_buffer.T @ x_optim)# + lam*cvx.sum(workload_mat @ x - w))
    constraints = [workload_mat[0] @ x_optim >= w[0]-(1-load[0])*horizon,
                   workload_mat[1] @ x_optim >= w[1]-(1-load[1])*horizon,
                   #(1/(1-load) * workload_mat) @ x_optim <= original_min_drain_time - (j+1)*horizon,
                   #(1/(1-load) * workload_mat) @ x_optim <= min_drain_time-horizon,
                   #1/(1-load[1]) * workload_mat[1] @ x_optim <= min_drain_time
                  ]
    prob = cvx.Problem(objective, constraints)

    eff_cost = prob.solve(solver=eval('cvx.CPLEX'),warm_start=True)
    x_star = x_optim.value
    w_star = workload_mat @ x_star
    
    v_star = w_star - w
    print(w_star.ravel().astype(int))
    
    non_idling = (v_star < 1e-4).ravel().astype(int)
    
    #if np.all(v_star < 1e-4):
     #   non_idling = np.zeros_like(non_idling)
        
    #if new_min_drain_time >= original_min_drain_time:
     #   non_idling = np.ones_like(non_idling)
    #else:
     #   input(non_idling)
    #print(non_idling)
    #if non_idling[1] == 1 or activate:
     #   activate = True
      #  print(v_star.)
       # input()
    
    z = cvx.Variable((3, 1), nonneg=True)
    h = cvx.Variable(nonneg=True)

    
    #if x[2] > 0:

    penalty_nonidling_vec = non_idling[0]*cvx.abs(resource_1 @ z - 1) + non_idling[1]*cvx.abs(resource_2 @ z - 1)
    penalty_nonidling_vec = cvx.abs((workload_mat[1] @ buffer_processing_matrix) @ z + 1) #+ cvx.abs((workload_mat[1] @ buffer_processing_matrix) @ z + 1)
 #   else:
        
    #penalty_nonidling_vec = cvx.abs(resource_2 @ z - 1)
    

    penalty_nonidling = cvx.sum(penalty_nonidling_vec)
    
    kappa_w = 1e5 #0 if x[2] > x[0] else 1e3        

    obj_equation = cost_vec @ z #+ h*kappa_w#kappa_w * cvx.sum(workload_mat @ (x + (buffer_processing_matrix @ z + demand_rate) * horizon) - w_star)
    objective = cvx.Minimize(obj_equation)

    constraints = [
        # Resource constraint.
        resource_1 @ z <= 1,
        resource_2 @ z <= 1,
        # Nonnegative future state.
        #(workload_mat[0] @ buffer_processing_matrix) @ z == -1,
        #(workload_mat[1] @ buffer_processing_matrix) @ z == -1,
        x + (buffer_processing_matrix @ z + demand_rate) * horizon >= 0,
        workload_mat @ (x + (buffer_processing_matrix @ z + demand_rate) * horizon) <= w_star,
        #workload_mat @ (x + (buffer_processing_matrix @ z + demand_rate) * horizon) <= h
    ]
    #if v_star[1] < 1e-3:
     #   constraints.append((workload_mat[1] @ buffer_processing_matrix) @ z == -1)

    cvx.Problem(objective, constraints).solve(solver=eval('cvx.CPLEX'),warm_start=True)

    z_star = z.value
    
    x_new = x + ((buffer_processing_matrix @ z_star + demand_rate) * horizon)
    
    state_seq.append(x.ravel())
    cost_seq.append(cost_per_buffer.T @ x)
    action_seq.append(z_star.ravel())
    cost += cost_per_buffer.T @ ((x_new+x)/2 * horizon)
    x = x_new
    
    #print(x.astype("int").ravel())
    print(h.value)
    print()
    

In [ ]:
cost/1e6

In [ ]:
cost/1e6

In [ ]:
plt.figure(figsize=(16,6))
for i,seq in enumerate(np.array(state_seq).T):
    plt.plot(np.arange(len(seq))*horizon,seq,label=i)
plt.legend()

In [ ]:
workload_mat[0]

In [ ]:
np.testing.assert_allclose(5,5.5
                           
                           
                           
                           
                           
                           
                           
                           
                           
                           
                           1,rtol=0.003)

In [ ]:
min_drain_time

In [ ]:
cost/1e6

In [ ]:
a = set([1,2,3])
b = set([1,2,3])

In [ ]:
a.issubset(b)

In [ ]:
cost_1/cost

In [ ]:
cost_per_buffer.T @ ((buffer_processing_matrix @ z_star + demand_rate) * horizon),cost_per_buffer.T @ (x2-x)

In [ ]:
x

In [ ]:
x2.astype(int)

In [ ]:
objective

In [ ]:
cost_vec @ z_star + cost_per_buffer.T @ demand_rate

In [ ]:
z_star

In [ ]:
cost_per_buffer.T @ ((buffer_processing_matrix @ z_star + demand_rate) * horizon),cost_per_buffer.T @ (x2-x)

In [ ]:
x2.astype(int)

In [ ]:
cost_vec @ z_star + cost_per_buffer.T @ demand_rate

In [ ]:
z_star

In [ ]:
cost_per_buffer.T @ ((buffer_processing_matrix @ z_star + demand_rate) * horizon)

In [ ]:
x2.astype(int)

In [ ]:
cost_vec @ z_star

In [ ]:
z_star

In [ ]:
x.astype(int)

In [ ]:
cost_vec @ z_star

In [ ]:
resource_1 @ z_star - 1, kappa_w*(resource_1 @ z_star - 1)

In [ ]:
5000*0.5*500000/cost

In [ ]:
plt.figure(figsize=(16,6))
for i,seq in enumerate(np.array(state_seq).T):
    plt.plot(np.arange(len(seq))*horizon,seq,label=i)
plt.legend()

In [ ]:
plt.plot(np.array(state_seq).T[2,20:50])
plt.ylim(top=1,bottom=-1)

In [ ]:
min_drain_time/15000

In [ ]:
min_drain_time

In [ ]:
plt.figure(figsize=(16,6))
for i,seq in enumerate(np.array(state_seq).T):
    plt.plot(np.arange(len(seq[:10]))*horizon,seq[:10],label=i)
plt.legend()

In [ ]:
original_min_drain_time

In [ ]:
plt.figure(figsize=(16,6))
for i,seq in enumerate(np.array(state_seq).T):
    plt.plot(np.arange(len(seq))*horizon,seq,label=i)
plt.legend()

In [ ]:
plt.figure(figsize=(16,6))
for i,seq in enumerate(np.array(state_seq).T):
    plt.plot(np.arange(len(seq))*horizon,seq,label=i)
plt.legend()

In [ ]:
plt.figure(figsize=(16,6))
for i,seq in enumerate(np.array(state_seq).T):
    plt.plot(np.arange(len(seq[:10]))*horizon,seq[:10],label=i)
plt.legend()

In [ ]:
plt.figure(figsize=(16,6))
for i,seq in enumerate(np.array(state_seq).T):
    plt.plot(np.arange(len(seq[:100]))*horizon,seq[:100],label=i)
plt.legend()

In [ ]:
plt.figure(figsize=(16,6))
for i,seq in enumerate(np.array(state_seq).T):
    plt.plot(np.arange(len(seq))*horizon,seq,label=i)
plt.legend()

In [ ]:
for i,seq in enumerate(np.array(state_seq).T):
    plt.plot(seq,label=i)
plt.legend()

In [ ]:
plt.plot(resource_2 @ np.array(action_seq).T)

In [ ]:
for i,seq in enumerate(np.array(action_seq).T):
    plt.plot(seq,label=i)
plt.legend()
plt.xlim(right=200)

In [ ]:
#cost1 = np.array(cost_seq).ravel()

In [ ]:
plt.plot(np.array(cost_seq).ravel())

In [ ]:
#plt.plot(np.arange(len(cost1))*10,np.array(cost1).ravel())
plt.plot(np.array(cost_seq).ravel())
#plt.xlim(right=50,left=-1)
#plt.ylim(top=50000)

In [ ]:
z_star

In [ ]:
x_new = x + ((buffer_processing_matrix @ z_star + demand_rate) * horizon).astype(int)

In [ ]:
x_new.astype("int")

In [ ]:
cost_per_buffer.T @ (x_new-x)

In [ ]:
cost_per_buffer

In [ ]:
x0 = np.array([0,10,100])[:,np.newaxis]
w = workload_mat @ x0
print(w)

In [ ]:
x0 = np.array([0,50+1e-3,50])[:,np.newaxis]
w1 = workload_mat @ x0
print(w1-w)

In [ ]:
num_buffers = workload_mat.shape[1]
lam = 1e5
x = cvx.Variable((num_buffers, 1))
objective = cvx.Minimize(cost_per_buffer.T @ x)# + lam*cvx.sum(workload_mat @ x - w))
constraints = [workload_mat @ x == w,
                x >= np.zeros((num_buffers, 1))]#,cvx.sum(x) <= np.sum(x0)]
prob = cvx.Problem(objective, constraints)
eff_cost = prob.solve(solver=eval('cvx.ECOS'))
x_star = x.value
w_star = workload_mat @ x_star
print(np.round(eff_cost))

In [ ]:
np.round(eff_cost)

In [ ]:
np.round(x_star)

In [ ]:
c_bar = -constraints[0].dual_value
print(c_bar)

In [ ]:
c_bar

In [ ]:
np.round(eff_cost + c_bar.T @ (w1-w))

In [ ]:
w_star-w

In [ ]:
cost_per_buffer.T @ x0 - eff_cost

In [ ]:
eff_cost

In [ ]:
eff_cost

In [ ]:
np.sum(x_star)

In [ ]:
np.sum(x0)-np.sum(x_star)

In [ ]:
np.sum(x_star)

In [ ]:
x_star

In [ ]:
np.sum(x_star)

In [ ]:
eff_cost

In [ ]:
eff_cost

In [ ]:
0.1341

In [ ]:
(1-3)*m2

In [ ]:
(2-3*2)*m2

In [ ]:
(w1-w)

In [ ]:
-constraints[0].dual_value.T @ (w1-w)

In [ ]:
w_star